In [1]:
import torch

# 张量操作

In [3]:
x=torch.rand(3,4)
x

tensor([[0.7582, 0.0339, 0.7738, 0.9723],
        [0.1135, 0.8129, 0.2985, 0.8373],
        [0.9428, 0.1428, 0.8890, 0.1018]])

In [4]:
torch.max(x)

tensor(0.9723)

In [5]:
torch.max(x,dim=1)

(tensor([0.9723, 0.8373, 0.9428]), tensor([3, 3, 0]))

In [10]:
loss = torch.rand(3,6)
_, indexes = loss.sort(dim=1, descending=True)
_, orders = indexes.sort(dim=1)

In [11]:
loss,indexes,orders

(tensor([[0.9049, 0.4682, 0.5858, 0.0140, 0.2309, 0.0964],
         [0.8476, 0.7515, 0.8630, 0.0433, 0.9716, 0.3245],
         [0.1229, 0.4248, 0.6381, 0.5952, 0.0086, 0.7843]]),
 tensor([[0, 2, 1, 4, 5, 3],
         [4, 2, 0, 1, 5, 3],
         [5, 2, 3, 1, 0, 4]]),
 tensor([[0, 2, 1, 5, 3, 4],
         [2, 3, 1, 5, 0, 4],
         [4, 3, 1, 2, 5, 0]]))

In [5]:
indexes,orders

tensor([[0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1]])

#  网络

## hook


pytorch 的 hook 函数分为 torch.Tensor 和 torch.nn.Module 两类， 分别对应

`torch.Tensor.register_hook
torch.nn.Module.register_backward_hook`

hook(module, grad_input, grad_output) -> Tensor or None


In [22]:
import torch
 
grad_list = []
 
def print_grad(grad):
    grad_list.append(grad)
    print(grad_list)
 
 
x = torch.randn(2, 1, requires_grad=True)
y = x + 2
z = torch.mean(torch.pow(y, 2))
y.register_hook(print_grad)    # 这里要注意， register_hook 接收的是一个函数
z.backward()


[tensor([[0.7182],
        [1.0360]])]


In [68]:
feat_result  = []
grad_result = []

def get_features_hook(self,input,output):
    # number of input:
    print('len(input): ',len(input))
    # number of output:
    print('len(output): ',len(output))
    print('###################################')
    print(input[0].shape) # torch.Size([1, 3, 224, 224])

    print('###################################')
    print(output[0].shape) # torch.Size([64, 55, 55])


    feat_result.append(output.data.cpu().numpy())

def get_grads_hook(self,input_grad, output_grad):
    # number of input:
    print('len(input): ', len(input_grad))
    # number of output:
    print('len(output): ', len(output_grad))

    print('###################################')
    print(input_grad[0]) # None 
    print(input_grad[1].shape) # torch.Size([64, 3, 11, 11]) for weights
    print(input_grad[2].shape) # torch.Size([64]) for bias

    print('###################################')
    print(output_grad[0].shape) # torch.Size([1, 64, 55, 55]) for x
    print('###################################')

    grad_result.append(output_grad[0].data.cpu().numpy())

In [69]:
import torch as t
import torch.nn as nn
import torch.nn.functional as F
   
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet,self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
   
    def forward(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = x.view(x.size()[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = LeNet()
img = torch.randn(1,1,32,32)
net(img)

tensor([[ 0.0025, -0.0792, -0.0684, -0.0508,  0.0497, -0.0340, -0.1051,  0.0418,
          0.0337,  0.0762]], grad_fn=<AddmmBackward>)

In [70]:
def hook(module, inputdata, output):
    '''把这层的输出拷贝到features中'''
    print(len(output))
    print(len(inputdata))
    print(torch.mean(output))
    print(torch.std(output))
 
handle = net.conv1.register_forward_hook(hook)
#handle2 = net.conv2.register_forward_hook(hook)
#handle3 = net.fc1.register_forward_hook(hook)
handle4 = net.fc1.register_backward_hook(hook)

loss=net(img)




1
1
tensor(-0.0480, grad_fn=<MeanBackward1>)
tensor(0.5992, grad_fn=<StdBackward0>)


In [71]:
loss=torch.sum(loss)
loss

tensor(-0.1335, grad_fn=<SumBackward0>)

In [72]:
loss.backward()


1
3


TypeError: mean(): argument 'input' (position 1) must be Tensor, not tuple

In [44]:
import torch
from torch.autograd import Variable
from torch.nn import Parameter
import torch.nn as nn
import math
def bh(m,gi,go):
    print("Grad Input")
    print(gi)
    print("Grad Output")
    print(go)
    return gi[0]*0,gi[1]*0
class Linear(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super(Linear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.Tensor(out_features, in_features))
        if bias:
            self.bias = Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input):
        if self.bias is None:
            return self._backend.Linear()(input, self.weight)
        else:
            return self._backend.Linear()(input, self.weight, self.bias)

x=Variable(torch.FloatTensor([[1, 2, 3]]),requires_grad=True)
mod=Linear(3, 1, bias=False)
mod.register_backward_hook(bh) # 在这里给module注册了backward hook

out=mod.forward(x)
out.register_hook(lambda grad: 0.1*grad) #在这里给variable注册了 hook
out.backward()
print(['*']*20)
print("x.grad", x.grad)
print(mod.weight.grad)

NotImplementedError: 

## 加载部分与训练模型

In [1]:
from torchvision import models
import torch
m = models.vgg16(pretrained=False)
f = 'D:/pretained_weight/vgg16-397923af.pth'
m.load_state_dict(torch.load(f))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [2]:
m

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

In [3]:

features = list(m.features)[:30]
features

[Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace),
 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace),
 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace),
 Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace),
 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace),
 Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace),
 Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace),
 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace),
 Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
 ReLU(inplace),
 

In [5]:
m = models.shufflenet_v2_x1_0(pretrained=False)
m

ShuffleNetV2(
  (conv1): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (stage2): Sequential(
    (0): InvertedResidual(
      (branch1): Sequential(
        (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=24, bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2d(24, 58, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(58, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace)
      )
      (branch2): Sequential(
        (0): Conv2d(24, 58, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(58, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True

In [8]:
a=list(m.children())[:4]
from torch import nn
import torchsummary
a=nn.Sequential(*a)
torchsummary.summary(a,(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 24, 112, 112]             648
       BatchNorm2d-2         [-1, 24, 112, 112]              48
              ReLU-3         [-1, 24, 112, 112]               0
         MaxPool2d-4           [-1, 24, 56, 56]               0
            Conv2d-5           [-1, 24, 28, 28]             216
       BatchNorm2d-6           [-1, 24, 28, 28]              48
            Conv2d-7           [-1, 58, 28, 28]           1,392
       BatchNorm2d-8           [-1, 58, 28, 28]             116
              ReLU-9           [-1, 58, 28, 28]               0
           Conv2d-10           [-1, 58, 56, 56]           1,392
      BatchNorm2d-11           [-1, 58, 56, 56]             116
             ReLU-12           [-1, 58, 56, 56]               0
           Conv2d-13           [-1, 58, 28, 28]             522
      BatchNorm2d-14           [-1, 58,

In [9]:
x=torch.randn(3,3)
x

tensor([[ 0.2704,  0.1428, -3.1219],
        [-1.6443,  2.6012, -0.4038],
        [ 0.0793, -0.6727,  0.5646]])

In [26]:
x[x>0]

tensor([0.2704, 0.1428, 2.6012, 0.0793, 0.5646])

In [31]:
ix=(x>0).nonzero()
x[ix[:,0],ix[:,1]],ix

(tensor([0.2704, 0.1428, 2.6012, 0.0793, 0.5646]), tensor([[0, 0],
         [0, 1],
         [1, 1],
         [2, 0],
         [2, 2]]))

In [33]:
torch.sum(x>0).item()

5

In [34]:
x.dim(),x.new().dim()

(2, 1)

In [25]:
x.new(1).dim()

1

In [15]:
x.new()[:,None,None]

tensor([], size=(0, 1, 1))

In [ ]:
from torchvision import ops
ops.box_iou
